# 🤖 Day 1-00.04: 모델 설정하기 (초보자용)

## 🎯 이번 노트북에서 할 일
- **EXAONE 모델** 로드하기 (Hugging Face 활용)
- **토크나이저** 설정하기
- **LoRA 설정**하기 (매우 간단하게!)
- **모델 준비** 완료하기

## 💡 사용할 모델
- **모델명**: `LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct`
- **언어**: 한국어에 특화
- **크기**: 7.8B 파라미터 (적당한 크기)
- **용도**: 질문답변, 대화, 텍스트 생성

## 🔧 LoRA란?
**Low-Rank Adaptation**의 줄임말로, 적은 메모리로 효율적으로 파인튜닝하는 방법입니다.

### 🎯 LoRA의 장점
- **메모리 절약**: 전체 모델을 학습하지 않고 일부만 학습
- **빠른 학습**: 적은 파라미터만 업데이트
- **안정적**: 기존 모델의 성능을 유지하면서 특화


## 1. 필요한 라이브러리 불러오기


In [ ]:
# 모델 설정에 필요한 라이브러리들을 불러옵니다
import torch
import json
from transformers import (
    AutoTokenizer,           # 텍스트를 토큰으로 변환
    AutoModelForCausalLM,    # 언어 모델
    BitsAndBytesConfig       # 4비트 양자화 설정
)
from peft import (
    LoraConfig,              # LoRA 설정
    get_peft_model,          # LoRA 모델 생성
    TaskType,                # 태스크 타입
    prepare_model_for_kbit_training  # 4비트 학습 준비
)
from datasets import load_from_disk
import warnings
warnings.filterwarnings('ignore')

print("✅ 모델 설정 라이브러리가 준비되었습니다!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"🚀 CUDA 사용 가능: {torch.cuda.is_available()}")


## 2. 모델 설정 (매우 간단하게!)


In [ ]:
# EXAONE 모델을 설정합니다 (하드코딩 없이!)
print("🤖 EXAONE 모델 설정 중...")

# 모델 이름 (Hugging Face에서 가져오기) - 공개 모델 사용
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

print(f"📥 모델 다운로드 중: {model_name}")
print("💡 이 과정은 처음에만 시간이 걸립니다. 다음에는 캐시에서 빠르게 로드됩니다.")

# 4비트 양자화 설정 (메모리 절약을 위해)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # 4비트로 로드
    bnb_4bit_compute_dtype=torch.float16, # 계산은 16비트로
    bnb_4bit_use_double_quant=True,       # 이중 양자화 사용
    bnb_4bit_quant_type="nf4"             # 4비트 양자화 타입
)

print("✅ 모델 설정 완료!")
print("   - 4비트 양자화: 메모리 절약")
print("   - 16비트 계산: 정확도 유지")
print("   - 이중 양자화: 더 나은 압축")


## 3. 토크나이저 로드하기


In [ ]:
# EXAONE-3.5-2.4B-Instruct 모델 사용 (공개 모델)
print("✅ EXAONE-3.5-2.4B-Instruct 모델 사용")
print("   - 공개 모델: 인증 불필요")
print("   - 크기: 2.4B 파라미터 (7.8B보다 작고 빠름)")
print("   - 한국어 특화: EXAONE 시리즈")


In [ ]:
# 토크나이저를 로드합니다 (텍스트를 토큰으로 변환하는 도구)
print("🔤 토크나이저 로드 중...")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,  # EXAONE 모델의 특별한 코드 사용
    padding_side="right"     # 패딩을 오른쪽에 추가
)

# 패딩 토큰 설정 (배치 처리에 필요)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # EOS 토큰을 패딩 토큰으로 사용

print("✅ 토크나이저 로드 완료!")
print(f"   - 어휘 크기: {tokenizer.vocab_size:,}개")
print(f"   - 패딩 토큰: {tokenizer.pad_token}")
print(f"   - EOS 토큰: {tokenizer.eos_token}")

# 토크나이저 테스트
test_text = "안녕하세요! 인공지능에 대해 설명해주세요."
tokens = tokenizer(test_text, return_tensors="pt")
print(f"\n🧪 토크나이저 테스트:")
print(f"   입력: {test_text}")
print(f"   토큰 수: {tokens['input_ids'].shape[1]}개")
print(f"   토큰 ID: {tokens['input_ids'][0][:10].tolist()}...")


## 4. 모델 로드하기


In [ ]:
# EXAONE 모델을 로드합니다 (4비트 양자화 적용)
print("🤖 EXAONE 모델 로드 중...")
print("💡 이 과정은 시간이 걸릴 수 있습니다. 잠시만 기다려주세요!")

# 모델 로드 (4비트 양자화 적용)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,  # 4비트 양자화 적용
    device_map="auto",                       # 자동으로 GPU/CPU 배치
    trust_remote_code=True,                  # EXAONE 모델의 특별한 코드 사용
    torch_dtype=torch.float16                # 16비트 부동소수점 사용
)

print("✅ 모델 로드 완료!")
print(f"   - 모델 크기: 7.8B 파라미터")
print(f"   - 양자화: 4비트 (메모리 절약)")
print(f"   - 디바이스: {next(model.parameters()).device}")

# 모델 정보 출력
print(f"\n📊 모델 정보:")
print(f"   - 모델 타입: {type(model).__name__}")
print(f"   - 학습 가능한 파라미터: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}개")
print(f"   - 전체 파라미터: {sum(p.numel() for p in model.parameters()):,}개")


## 5. LoRA 설정하기 (매우 간단하게!)


In [ ]:
# LoRA 설정을 만듭니다 (모든 파라미터를 설명!)
print("🔧 LoRA 설정 중...")

# LoRA 설정 (각 파라미터의 의미를 명확히 설명)
lora_config = LoraConfig(
    r=16,                           # LoRA rank (낮을수록 빠름, 높을수록 정확)
    lora_alpha=32,                  # LoRA scaling (학습률 조절)
    target_modules=[                # 어떤 레이어를 학습할지
        "q_proj",                   # Query 프로젝션 (어텐션의 질문 부분)
        "v_proj",                   # Value 프로젝션 (어텐션의 값 부분)
        "k_proj",                   # Key 프로젝션 (어텐션의 키 부분)
        "o_proj"                    # Output 프로젝션 (어텐션 출력)
    ],
    lora_dropout=0.05,              # 과적합 방지를 위한 드롭아웃
    bias="none",                    # 바이어스 학습 안함
    task_type=TaskType.CAUSAL_LM,   # 언어 모델링 태스크
)

print("✅ LoRA 설정 완료!")
print("   - r=16: LoRA rank (적당한 크기)")
print("   - alpha=32: 스케일링 팩터")
print("   - target_modules: 어텐션 레이어들")
print("   - dropout=0.05: 과적합 방지")
print("   - task_type: 언어 모델링")


## 6. LoRA 모델 생성하기


In [ ]:
# 4비트 학습을 위한 모델 준비
print("🔄 4비트 학습 준비 중...")

# 4비트 양자화된 모델을 학습 가능하게 준비
model = prepare_model_for_kbit_training(model)

# LoRA 어댑터를 모델에 추가
print("🔧 LoRA 어댑터 추가 중...")
model = get_peft_model(model, lora_config)

print("✅ LoRA 모델 생성 완료!")

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print(f"\n📊 파라미터 정보:")
print(f"   - 학습 가능한 파라미터: {trainable_params:,}개")
print(f"   - 전체 파라미터: {all_params:,}개")
print(f"   - 학습 비율: {trainable_params/all_params*100:.2f}%")
print(f"   - 메모리 절약: {(1-trainable_params/all_params)*100:.1f}%")

# 모델 구조 확인
print(f"\n🔍 LoRA 어댑터 정보:")
for name, module in model.named_modules():
    if hasattr(module, 'lora_A') and hasattr(module, 'lora_B'):
        print(f"   - {name}: LoRA 어댑터 적용됨")


## 7. 모델 테스트하기


In [ ]:
# 설정된 모델이 제대로 작동하는지 테스트해봅시다
print("🧪 모델 테스트 중...")

# 테스트 텍스트
test_prompt = "질문: 인공지능이란 무엇인가요?\n답변:"

# 토크나이징
inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

# 생성 설정
generation_config = {
    "max_new_tokens": 50,        # 최대 50개 토큰 생성
    "temperature": 0.7,          # 창의성 조절 (0.7 = 적당히 창의적)
    "do_sample": True,           # 샘플링 사용
    "pad_token_id": tokenizer.eos_token_id  # 패딩 토큰 ID
}

# 텍스트 생성
with torch.no_grad():  # 그래디언트 계산 안함 (메모리 절약)
    outputs = model.generate(
        **inputs,
        **generation_config
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("✅ 모델 테스트 완료!")
print(f"\n📝 테스트 결과:")
print(f"입력: {test_prompt}")
print(f"출력: {generated_text}")
print(f"\n🎯 생성된 부분만:")
print(generated_text[len(test_prompt):])


## 8. 설정 저장하기


In [ ]:
# 모델 설정을 저장합니다 (다음 단계에서 사용)
print("💾 모델 설정 저장 중...")

# 모델 설정 정보
model_config = {
    "model_name": model_name,
    "lora_config": {
        "r": lora_config.r,
        "lora_alpha": lora_config.lora_alpha,
        "target_modules": lora_config.target_modules,
        "lora_dropout": lora_config.lora_dropout,
        "bias": lora_config.bias,
        "task_type": str(lora_config.task_type)
    },
    "quantization_config": {
        "load_in_4bit": quantization_config.load_in_4bit,
        "bnb_4bit_compute_dtype": str(quantization_config.bnb_4bit_compute_dtype),
        "bnb_4bit_use_double_quant": quantization_config.bnb_4bit_use_double_quant,
        "bnb_4bit_quant_type": quantization_config.bnb_4bit_quant_type
    },
    "model_info": {
        "trainable_params": int(trainable_params),
        "all_params": int(all_params),
        "trainable_ratio": float(trainable_params/all_params)
    }
}

# 설정 저장
with open("models/model_config.json", "w", encoding="utf-8") as f:
    json.dump(model_config, f, ensure_ascii=False, indent=2)

# LoRA 설정도 별도로 저장
model.save_pretrained("models/lora_config")

print("✅ 설정 저장 완료!")
print("   - models/model_config.json: 전체 설정")
print("   - models/lora_config/: LoRA 설정")
print("   - 다음 단계에서 이 설정들을 사용합니다")


## 9. 다음 단계 안내

### 🎯 다음 노트북에서 할 일
**00.05-fine-tuning.ipynb**에서:
1. **RAFT 데이터** 로드하기
2. **학습 설정** 구성하기
3. **실제 파인튜닝** 실행하기
4. **학습 과정** 모니터링하기

### 💡 지금까지 배운 것
- ✅ EXAONE 모델 로드 및 설정
- ✅ 4비트 양자화로 메모리 절약
- ✅ LoRA 설정으로 효율적 학습
- ✅ 토크나이저 설정 및 테스트
- ✅ 모델 설정 저장

### 🔧 LoRA의 핵심
- **r=16**: LoRA rank (적당한 크기)
- **target_modules**: 어텐션 레이어들만 학습
- **메모리 절약**: 전체 모델의 1%만 학습
- **빠른 학습**: 적은 파라미터로 효율적

### 🚀 준비 완료!
이제 다음 노트북으로 넘어가서 실제 파인튜닝을 실행해보겠습니다!
